In [1]:
import pandas as pd
import numpy as np

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../data/voice_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../data/sms_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../data/wa_test_a.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
prefix = np.array(['u'])
uid_num = np.arange(5000,7000)
uid_num_char = uid_num.astype('U')
uid_num_str = np.core.defchararray.add(prefix, uid_num_char)
uid_test = pd.DataFrame(uid_num_str, columns=['uid'])
uid_test.to_csv('../data/uid_test_a.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

### 通话记录

In [6]:
voice['voice_dura']=abs(voice.end_time.astype('int')-voice.start_time.astype('int'))
# 不同的电话号码数/电话总数
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

# 不同的电话号码头三位的数量
voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

# 每种电话长度的通话次数
voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

# 每种类型通话的次数
voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

# 每种类型通话的平均时长
voice_dura_type = voice.groupby(['uid','call_type'])['voice_dura'].mean().unstack().add_prefix('voice_dura_type_').reset_index().fillna(0)

# 接入/打出的电话总数
voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

# 通话时长的各统计量
voice_dura = voice.groupby(['uid'])['voice_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_dura_').reset_index().fillna(0)

## 每个用户收/发电话的号码的不同号码数
voice_opp_len_inout = voice.groupby(['uid','in_out'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).unstack().add_prefix('voice_opp_inout_num_').reset_index().fillna(0)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [7]:
voice_dura_type.head()

call_type,uid,voice_dura_type_1,voice_dura_type_2,voice_dura_type_3,voice_dura_type_4,voice_dura_type_5
0,u0001,451.721519,0.0,0.000000,0.0,0.0
1,u0002,49.500000,0.0,0.000000,0.0,0.0
2,u0003,134.619048,0.0,0.000000,0.0,0.0
3,u0004,547.229358,1628.0,950.833333,0.0,0.0
4,u0005,132.876884,0.0,82.333333,0.0,0.0


## 短信记录

In [8]:
# 不同的短信号码数/电话总数
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()

# 号码不同头三位的数量
sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

# 每种号码长度的通话次数
sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

# 接受/发出短信总数
sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


### 网站/APP记录

In [9]:
# 不同wa数量/wa总数
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()

# 访问wa次数的各统计量
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

# 访问wa时长的各统计量
visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()

# 上传流量的各统计量
up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

# 下载流量的各统计量
down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

## 每个用户上/下的流量
w_a_upflow = wa.groupby(['uid','wa_type'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_upflow_').unstack().reset_index().fillna(0)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [10]:
w_a_upflow.head()

uid w_a_upflow_std               w_a_upflow_max              \
wa_type                   0.0           1.0            0.0         1.0   
0        u0001  785782.118695  5.426537e+05     10720264.0   3360841.0   
1        u0002    5429.253721  0.000000e+00        22319.0         0.0   
2        u0003  183276.260009  7.804732e+05      3709153.0   8915918.0   
3        u0004  924427.079979  9.353514e+05     27479806.0   7399037.0   
4        u0005  126144.674813  1.358326e+06      1422573.0  12441601.0   

        w_a_upflow_min       w_a_upflow_median          w_a_upflow_mean  \
wa_type            0.0   1.0               0.0      1.0             0.0   
0                  0.0   0.0            5563.0   9788.5   168850.068063   
1                  0.0   0.0            2453.0      0.0     4433.551724   
2                  0.0   0.0            3469.0   4784.0    37904.870672   
3                  0.0   0.0            4416.0  48402.5   129466.396501   
4                  0.0  40.0            4040.0   4595.0    38306.330341   

                       w_a_upflow_sum               
wa_type            1.0            0.0          1.0  
0        188654.245000    161251815.0   37730849.0  
1             0.000000       128573.0          0.0  
2        182601.945946     37222583.0   33781360.0  
3        414593.694805    177627896.0  127694858.0  
4        228585.511905     21336626.0   19201183.0

In [11]:
# 将各个特征拼在一起
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,voice_dura,voice_opp_len_inout,voice_dura_type,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
           down_flow,w_a_upflow]

In [12]:
# 将train特征和test特征分离
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

D:\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Anaconda\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [13]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

D:\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Anaconda\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [14]:
# 填补Nan
train_feature.fillna(0,inplace=True)
test_feature.fillna(0,inplace=True)

In [15]:
# 保存特征值
train_feature.to_csv('../data/train_featureV1.csv',index=None)
test_feature.to_csv('../data/test_featureV1.csv',index=None)

In [16]:
train_feature.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,...,"(w_a_upflow_max, 0.0)","(w_a_upflow_max, 1.0)","(w_a_upflow_min, 0.0)","(w_a_upflow_min, 1.0)","(w_a_upflow_median, 0.0)","(w_a_upflow_median, 1.0)","(w_a_upflow_mean, 0.0)","(w_a_upflow_mean, 1.0)","(w_a_upflow_sum, 0.0)","(w_a_upflow_sum, 1.0)"
0,u0001,0,22.0,79.0,17.0,0.0,3.0,0.0,0.0,0.0,...,10720264.0,3360841.0,0.0,0.0,5563.0,9788.5,168850.068063,188654.245000,161251815.0,37730849.0
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,22319.0,0.0,0.0,0.0,2453.0,0.0,4433.551724,0.000000,128573.0,0.0
2,u0003,0,15.0,21.0,10.0,0.0,1.0,0.0,0.0,2.0,...,3709153.0,8915918.0,0.0,0.0,3469.0,4784.0,37904.870672,182601.945946,37222583.0,33781360.0
3,u0004,0,77.0,254.0,31.0,0.0,1.0,0.0,0.0,12.0,...,27479806.0,7399037.0,0.0,0.0,4416.0,48402.5,129466.396501,414593.694805,177627896.0,127694858.0
4,u0005,0,55.0,401.0,28.0,0.0,4.0,0.0,0.0,0.0,...,1422573.0,12441601.0,0.0,40.0,4040.0,4595.0,38306.330341,228585.511905,21336626.0,19201183.0
